## This is the original file from minGPT - but have added sections to specifically discuss about GPU training aspects and some of the challenges. gptMin code base is used as back drop in doing this demo

## This is the original repo - https://github.com/karpathy/minGPT

## Train a character-level GPT on some text data

The inputs here are simple text files, which we chop up to individual characters and then train GPT on. So you could say this is a char-transformer instead of a char-rnn. Doesn't quite roll off the tongue as well. In this example we will feed it some Shakespeare, which we'll get it to predict character-level.

In [1]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [4]:
import math
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y


In [23]:
# Language model will actually make block_size individual predictions at the same time based on this data
    
block_size = 128 # 128 spatial extent of the model for its context

In [7]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2020-09-12 23:14:38--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.0.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2020-09-12 23:14:38 (15.7 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [12]:
# you can download this file at https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt
text = open('input.txt', 'r').read() # don't worry we won't run out of file handles
train_dataset = CharDataset(text, block_size) # one line of poem is roughly 50 characters

data has 1115394 characters, 65 unique.


In [16]:
from mingpt.model import GPT, GPTConfig
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=512)
model = GPT(mconf)

09/15/2020 13:52:56 - INFO - mingpt.model -   number of parameters: 2.535219e+07


In [56]:
torch.cuda.max_memory_cached(0)
# 2 GPUs max_memory_cached(0) is 7637827584
# div by 10e9 to GB

10729029632

In [55]:
torch.cuda.empty_cache()

In [53]:
torch.cuda.max_memory_allocated(0)
# 2 GPUs max_memory_allocated(0) = 7621877760

10727858176

In [47]:
torch.cuda.memory_allocated(0)
# 2 GPUs cuda.memory_allocated(0)= 417400320

10727858176

In [48]:
torch.cuda.memory_cached(0)
# 2 GPUs cuda.memory_cached(0) = 7054819328

10729029632

In [49]:
# cached minus allocated free memory
torch.cuda.memory_cached(0) - torch.cuda.memory_allocated(0)

1171456

In [44]:
# This script looks up memory usage across all the gpu machines
# courtesy of mjstevens777 Matt

import subprocess

def get_gpu_memory_map():
    """Get the current gpu usage.

    Returns
    -------
    usage: dict
        Keys are device ids as integers.
        Values are memory usage as integers in MB.
    """
    result = subprocess.check_output(
        [
            'nvidia-smi', '--query-gpu=memory.used',
            '--format=csv,nounits,noheader'
        ], encoding='utf-8')
    # Convert lines into a dictionary
    gpu_memory = [int(x) for x in result.strip().split('\n')]
    gpu_memory_map = dict(zip(range(len(gpu_memory)), gpu_memory))
    return gpu_memory_map
get_gpu_memory_map()
#{0: 7326, 1: 6992, 2: 6992, 3: 6992}

{0: 11014}

In [43]:
# prints currently alive Tensors and Variables
# courtesy Smth, PyTorch Dev, Facebook AI Research
# use the garbage collector’s book-keeping 
# to print out the currently resident Tensors. 
# Here’s a snippet that shows all the currently allocated Tensors:

import torch
import gc
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size())
    except:
        pass

<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size([512, 128])
<class 'torch.Tensor'> torch.Size(

/opt/conda/lib/python3.7/site-packages/torch/distributed/distributed_c10d.py:102: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


In [42]:
# make sure you run this pip install pynvml

from pynvml import *
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total}')
print(f'free     : {info.free}')
print(f'used     : {info.used}')

# total    : 8370061312
# free     : 687865856
# used     : 7682195456
    

total    : 11554717696
free     : 5373952
used     : 11549343744


In [33]:
!pip install GPUtil

  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=5bae5a2c7cae3f3a20711482acc9f3d0acc279ebe6b09cb95cedba078590c316
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [41]:
# make sure to install GPUtil before running this utility

import torch
from GPUtil import showUtilization as gpu_usage

print("Initial GPU Usage")
gpu_usage()                             

tensorList = []
#for x in range(10):
#    tensorList.append(torch.randn(10000000,10).cuda())   # reduce the size of tensor if you are getting OOM

print("GPU Usage after allcoating a bunch of Tensors")
gpu_usage()

del tensorList

print("GPU Usage after deleting the Tensors")
gpu_usage()  

print("GPU Usage after emptying the cache")
torch.cuda.empty_cache()
gpu_usage()

Initial GPU Usage
| ID | GPU | MEM  |
-------------------
|  0 |  0% | 100% |
GPU Usage after allcoating a bunch of Tensors
| ID | GPU | MEM  |
-------------------
|  0 |  0% | 100% |
GPU Usage after deleting the Tensors
| ID | GPU | MEM  |
-------------------
|  0 |  0% | 100% |
GPU Usage after emptying the cache
| ID | GPU | MEM  |
-------------------
|  0 |  0% | 100% |


In [38]:
# better to run this in terminal
!watch -n 1 free -m

>p:655350655357258425020.1: Sun Sep 13 05:12:57 2020totalusedfreeshared  buff/cache   available8929193:0041828354564997858891017723884567892049912345678988303302012345642818799894078150519529849882363330502045501965020729198950588123445677673998944:00881239934453206230811910527171230202345637298309291920844158824357475667859308817273456788974081728347112595297498864876332950208950195085192348847556487785970125:001121342051126170892103190122334253096750108480892066261715728344589677889530712384458677589740128345767899296498750934164328501827354758674857986:00418234577548867789810912835774856767789820489182399348547868739899304881392345486739849394048812349563974848899503912929499034989465327501853996748937:001482394856879839948108192835744856676757892012345488679830677149928384557678

In [35]:
# how to check your GPU device names
torch.cuda.get_device_name(0)

'GeForce RTX 2080 Ti'

In [36]:
#Tracking Memory Usage with GPUtil


!pip install GPUtil
import GPUtil
GPUtil.showUtilization()

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [40]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=1, batch_size=64, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                      num_workers=4)
trainer = Trainer(model, train_dataset, None, tconf)
trainer.train()




  0%|          | 0/17427 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 10.76 GiB total capacity; 9.99 GiB already allocated; 5.12 MiB free; 9.99 GiB reserved in total by PyTorch)

## GPU Type and number of instances - 
### 'GeForce RTX 2080 Ti' - 1 instance - batch size 512
#### CUDA OOM; Tried to allocate 256.00 MiB (GPU 0; 10.76 GiB total capacity; 9.76 GiB already allocated; 135.12 MiB free; 9.87 GiB reserved in total by PyTorch)

### 'GeForce RTX 2080 Ti' - 1 instance - batch size 256
#### CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 10.76 GiB total capacity; 9.96 GiB already allocated; 5.12 MiB free; 9.99 GiB reserved in total by PyTorch)

### 'GeForce RTX 2080 Ti' - 1 instance - batch size 128
#### CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 10.76 GiB total capacity; 9.99 GiB already allocated; 5.12 MiB free; 9.99 GiB reserved in total by PyTorch)


In [10]:
# alright, let's sample some character-level Shakespeare
from mingpt.utils import sample

context = "O God, O God!"
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, 2000, temperature=1.0, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

O God, O God! that e'er this tongue of mine,
That laid the sentence of dread banishment
On yon proud man, should take it off again
With words of sooth! O that I were as great
As is my grief, or lesser than my name!
Or that I could forget
With Richmond, I'll tell you what I am,
The Lord Aumerle, .

CLAUDIO:
The prenzie Angelo!

ISABELLA:
O, 'tis the cunning livery of hell,
The damned'st body to invest and cover
In prenzie guards! Dost thou think, Claudio?
If I would yield him my virginity,
Thou mightst be freed.

CLAUDIO:
O heavens! it cannot be.

ISABELLA:
Yes, he would give't thee, from this rank offence,
So to offend him still. This night's the time
That I should do what I abhor to name,
Or else thou diest to-morrow.

CLAUDIO:
Thou shalt not do't.

ISABELLA:
O, were it but my life,
I'ld throw it down for your deliverance
As frankly as a pin.

CLAUDIO:
Thanks, dear Isabel.

ISABELLA:
Be ready, Claudio, for your death tomorrow.

CLAUDIO:
Yes. Has he affections
That profit us.

DUKE VIN

In [ ]:
# well that was fun

# Final Comments 

### * You will notice that Python created folders with __pycache__ and also files with .pyc or .pyo extension. These are bytecode-compiled and optimized bytecode-compiled versions of your program's files, respectively. They make your program start a little faster. When your scripts change, they will be recompiled, and if you delete the files or the whole folder and run your program again, they will reappear (unless you specifically suppress that behavior). Just put them in your gitignore
